#**KumoRFM Demo Recommender System**

---

**Thank you for trying out KumoRFM!**  
This demo is built on top of a **comprehensive, detailed sample online dataset** and is designed to show how quickly you can go from raw data to actionable product recommendations.

---

> ⚠️ **Note:** While we’ve intentionally made this example look simple and seamless, **real-world implementations will vary**.  
> You may need to adapt:
> - Data preprocessing steps  
> - Feature engineering  
> - Model configuration  
> - Integration with your own data sources  
>
> One thing is certain — **no model training is required** to get started.

---

📚 **Learn More**  
For detailed guidance, advanced configurations, and inspiration for adapting KumoRFM to a variety of use cases, explore our  
[**Official Documentation →**](https://kumo-ai.github.io/kumo-sdk/docs/get_started/rfm/index.html)  
We strongly encourage you to experiment with the various features to tailor the system to your own needs.

---


## Step 1. Install the Kumo Python SDK and this project's requirements

KumoRFM provides an [SDK](https://kumo-ai.github.io/kumo-sdk/docs/get_started/rfm/index.html) in Python.
The Kumo SDK is available for Python 3.9 to Python 3.13. All of the files used in this document can be found [here](https://).

In [ ]:
!wget -O requirements.txt "https://drive.google.com/uc?export=download&id=14u26T_oxPBKGzoC7d9h6gtpkz5bgWzSV" # Installing the list of dependencies, and helper files.
!pip install -q gdown
!gdown --id 1WUszVhUN23tl1w0wmNYRTrLpvqKEuFum -O my_display.py

In [ ]:
!pip install -r requirements.txt # Installing the dependencies themselves.

In [ ]:
import pandas as pd #Import the tools we need
from kumoai.experimental import rfm
import os,dotenv
from my_display import display_recommendations # Our online store

## Step 2. Get an API key

You will need an API key to make calls to KumoRFM.
Use the widget below to generate one for free by clicking "Generate API Key".
If you don't have a KumoRFM account, the widget will prompt you to signup.

You will see the following when your key has been created successfully:

<div align="left">
  <img src="https://kumo-sdk-public.s3.us-west-2.amazonaws.com/rfm-colabs/api-key-created.png" width="300" />
</div>

In [ ]:
dotenv.load_dotenv()
if not os.getenv("KUMO_API_KEY"):
  rfm.authenticate()

## Step 3. Initialize a client

If you completed step 2 via the widget, you don't need to change anything. `KUMO_API_KEY` is already set as environment variable.

If you bring the API key from the website, you can manually change the `KUMO_API_KEY` below:


In [ ]:
# Initialize a Kumo client with your API key:
KUMO_API_KEY = os.getenv("KUMO_API_KEY")
rfm.init(api_key=KUMO_API_KEY)

## Step 4. Import your data

KumoRFM interacts with a set of `pd.DataFrame` objects:

In [ ]:
USER_CURRENT = 50  # This is me. You can set it to any user_id from 0 to 11,462.

root = "s3://rfm-shopping-recommender"
data = {
"users": pd.read_parquet(f"{root}/users.parquet"),
"items": pd.read_parquet(f"{root}/items.parquet"),
"views": pd.read_parquet(f"{root}/views.parquet"),
"orders": pd.read_parquet(f"{root}/orders.parquet"),
"returns": pd.read_parquet(f"{root}/returns.parquet"),
}

# NOTE You can use `pd.read_csv(...)` to read CSV files instead.
# You don't need to use s3 to import your data.
# For Colab users, you can upload your data to the file folder (the folder icon on the left panel), and directly import from there.
# You can also download this notebook, and import your data locally.

In [ ]:
#Feel free to inspect the data, or inspect its types
data["users"].head(5)
data["users"].dtypes

## Step 5. Spin up your model!

KumoRFM will create a `LocalGraph` from the data you imported above, and then initialize a model in just two simple lines of code.

KumoRFM is smart enough to infer the relations between your data tables using column names. You may still want to inspect the results of inferred metadata to ensure correctness downstream.

If your tables don't have matching names, you'll want to link them on your own. Refer to our documentation [here](https://kumo-ai.github.io/kumo-sdk/docs/get_started/rfm/index.html).

In [ ]:
graph = rfm.LocalGraph.from_data(data)
model = rfm.KumoRFM(graph)

## Step 6. Write a predictive query

You are now ready to plug your graph into `KumoRFM` to make predictions!

The great thing about the graph is that it's a one-time setup—once it's in place, you can generate a variety of predictions from it and power many business use cases.

#### PQueries: How you talk to your model

Learn more about writing ALL kinds of PQueries [here](https://kumo.ai/docs/pquery-structure/) for any prediction you might want.

In [ ]:
query = (f"PREDICT LIST_DISTINCT(orders.item_id, 0, 30, days) RANK TOP 10 FOR users.user_id={USER_CURRENT}")

# To break this down:

# PREDICT LIST_DISTINCT(orders.item_id, 0, 30, days) = Predict the list of distinct items most likely to be ordered in the next 30 days.
# RANK TOP 10 restricts the output to only the 10 most likely to be purchased items(i.e., the ones that should be recommended).
# FOR restricts the scope of the query to whatever USER_CURRENT is. Outputting results for multiple users at a time is possible.

# Mess around with it on your own! Refer to the documentaton above.

## Step 7. Predict away and display!

With a simple command, you can instantly predict based on your query above.

In [ ]:
pred_df = model.predict(query)

#View the prediction like this:
display(pred_df)

In [ ]:
#Or display it in the store, like this:

display_recommendations(pred_df, data["items"])
!streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py

# NOTE: Prediction might be slightly different here, since its rerunning this script to obtain it, and the output isn't perfectly deterministic.
# Before running other boxes, make sure to stop this one, since streamlit will prevent other boxes from being run while in use! :)